# Using the Datashader `HoverLayer`

This notebook reviews use of the `datashader.bokeh_ext.HoverLayer` which integrates `datashader` with the Bokeh `HoverTool`. 

Using HoverLayer, you can display values from a 'datashaded' image in Bokeh tooltips.

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import Figure

import pandas as pd
import datashader as ds
import datashader.transfer_functions as tf

from datashader.colors import Hot
from datashader.bokeh_ext import HoverLayer

output_notebook()

### Load NYC Taxi Data

In [ ]:
df = pd.read_csv('data/nyc_taxi.csv', usecols=['dropoff_x', 'dropoff_y'])

### Simple datashaded image displayed using Bokeh.ImageRGBA

In [ ]:
def create_base_plot():
    
    # taxi data is in meters
    xmin = -8240227.037
    ymin = 4974203.152
    xmax = -8231283.905
    ymax = 4979238.441

    cvs = ds.Canvas(plot_width=900,
                    plot_height=600,
                    x_range=(xmin, xmax),
                    y_range=(ymin, ymax))

    agg = cvs.points(df, 'dropoff_x', 'dropoff_y')
    img = tf.interpolate(agg, cmap=Hot, how='log')
    fig = Figure(x_range=(xmin, xmax),
                 y_range=(ymin, ymax),
                 plot_width=900,
                 plot_height=600,
                 tools='')
    
    fig.background_fill_color = 'black'
    fig.toolbar_location = None
    fig.axis.visible = False
    fig.grid.grid_line_alpha = 0
    fig.min_border_left = 0
    fig.min_border_right = 0
    fig.min_border_top = 0
    fig.min_border_bottom = 0

    fig.image_rgba(image=[img.data],
                   x=[xmin],
                   y=[ymin],
                   dw=[xmax-xmin],
                   dh=[ymax-ymin])
    return fig, (xmin, ymin, xmax, ymax), agg

fig, extent, datashader_agg = create_base_plot()
show(fig)


Above is a base datashaded image of NYC Taxi data as a starting place to explore HoverLayer.  We are visualizing the number of dropoffs at a given location. 

The simpliest hover layer has 3 required arguments, all of which return from the `create_base_plot` function:
- A datashader aggregate (xarray)
- An extent (xmin, ymin, xmax, ymax)
- The field name with the aggregate to summarize in the HoverLayer.


In [ ]:
fig, extent, datashader_agg = create_base_plot()

hover_layer = HoverLayer(agg=datashader_agg,
                         extent=extent,
                         field_name='Average Dropoff Count')

fig.renderers.append(hover_layer.renderer)
fig.add_tools(hover_layer.tool)
show(fig)

By default, the `HoverLayer` will use the `mean` of the values (excluding NaNs). When aggregating points, `Canvas.points` defaults to a `count`, thus the tooltips above show the Average Count of passengers.

Instead of showing the `Average Count`, let's display the `Total` count by setting `HoverLayer.how` to `sum`:

In [ ]:
fig, extent, datashader_agg = create_base_plot()

hover_layer = HoverLayer(agg=datashader_agg,
                         extent=extent,
                         field_name='Total Dropoffs',
                         how='sum')

fig.renderers.append(hover_layer.renderer)
fig.add_tools(hover_layer.tool)
show(fig)

Now that we have a basic hover function, let's change it size and selection glyph appearance:

In [ ]:
fig, extent, datashader_agg = create_base_plot()
hover_layer = HoverLayer(field_name='Total Dropoffs',
                         highlight_fill_color='#FFFFFF',
                         highlight_line_color='#FFFFFF',
                         size=30,
                         is_categorical=False,
                         extent=extent,
                         agg=datashader_agg,
                         how='sum')

fig.renderers.append(hover_layer.renderer)
fig.add_tools(hover_layer.tool)
show(fig)